##Start up

### GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Enable TPU
Edit --> Notebook Settings --> TPU
Then check if TPU works

Ensure that the number of instances is perfectly divisible by the steps_per_epoch parameter so that all the instances are used during
training. For example, we have 60000 instances in our training set. 60000
is divisible by 50 so that means all our instances are fed into the model without any leftovers. If you hit run, it should start training on a TPU instance after a short while:


Basically should have that data is 256 all the time in batch not a leftover with less... Could get rid of the data that does not fit into it?

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)


In [ ]:
#This strategy ssems to not work?

# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

### Import lib

In [ ]:
#Genna
!pip install -q segmentation_models



In [ ]:
import tensorflow as tf

In [ ]:
print (tf.version)

In [ ]:
from segmentation_models.losses import binary_focal_dice_loss #this also exists..

In [ ]:
from segmentation_models.losses import bce_dice_loss

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
# import tensorflow as tf
import pathlib
import re
import random
from tqdm.notebook import tqdm
import math
# import itertools
import time
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# import seaborn as sns

In [ ]:

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.optimizers import Adam #worked before since 19-08-2021 doesnt work anymore so now import via tf
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:

from skimage.io import imread
from skimage.transform import resize
# from keras.models import Sequence

In [ ]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
# from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
# from matplotlib import pyplot


## Choose properties

In [ ]:
# global parameters
P = {

    'SEED': 42,
     
    # image size to sample
    # 'SIZE': 160, 
    # 'SIZE': 224, 
     'SIZE': 256, 
    # 'SIZE': 320,
    # 'BATCH_SIZE': 32 * strategy.num_replicas_in_sync, #256.. 64 is 512. 
    
    # 'FOLDS': 5,
    
    # 'LEARNING_RATE': 0.00075,
    
    'EPOCHS': 100,
     
    # 'VERSION': 'v6'
}

In [ ]:
# BATCH_SIZE_PER_REPLICA = 64
# GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# tpu_strategy.num_replicas_in_sync #8.
# 256/8

In [ ]:
n_batch = 64
num_epochs = 100 #200 in og pix2pix recommended
path_save_gan = "/content/drive/MyDrive/Thesis/big_ls_augment_v3/temp_models/"
# os.makedirs(path_save_gan)
#attempt 1; repeat(2)
#attempt 2; repeat(3)


In [ ]:
!ls -la {path_save_gan}

In [ ]:
path_results = "/content/drive/MyDrive/Thesis/big_ls_augment_v3/Results/attempt_1_1409_bcedice_des_and_asc_bsize_" + str(n_batch) + "/"
os.makedirs(path_results)

In [ ]:
path_to_save = path_save_gan + "attempt_1_1409_bcedice_des_and_asc_bsize_" + str(n_batch) + "/" #hceck if works 
os.makedirs(path_to_save) #seems as if this is not made...

In [ ]:
path_results

In [ ]:
path_to_save

In [ ]:
def remove_files(dir):
  for f in os.listdir(dir):
      os.remove(os.path.join(dir, f))

In [ ]:
# remove_files(path_to_save)
# remove_files(path_results)

In [ ]:
!ls -la {path_to_save}
!ls -la {path_results}

files from kaggle

In [ ]:
tfrec_files_all_ls = tf.io.gfile.glob('gs://kds-60e70aece3307e40b9e154943a40dcbdb493d3d10f4776c24b1d0174/*')
print('all ls files',tfrec_files_all_ls)
tfrec_files_all_ls_des = tfrec_files_all_ls[6:]
print('des ls files',tfrec_files_all_ls_des)
tfrec_files_all_ls_asc = tfrec_files_all_ls[:6]
print('asc ls files',tfrec_files_all_ls_asc)


In [ ]:
tfrec_files_ls = tf.io.gfile.glob('gs://kds-2fce72c5034cc086f74d9f3987cda7e253d4f468e128d93560b9a47b/*')
print('all ls files',tfrec_files_ls)
tfrec_files_ls_des = tfrec_files_ls[3:]
print('des ls files',tfrec_files_ls_des)
tfrec_files_ls_asc = tfrec_files_ls[:3]
print('des asc val files',tfrec_files_ls_asc)

In [ ]:
tfrec_files_val_ls = tf.io.gfile.glob('gs://kds-edf2dfc9e536d4c9873bf35b19e5a84be62a4e2f30964f18939238dd/*')
print('all ls val files',tfrec_files_val_ls)
tfrec_files_val_ls_des =  tfrec_files_val_ls[2:]
print('des ls val files',tfrec_files_val_ls_des)
tfrec_files_val_ls_asc =  tfrec_files_val_ls[:2]
print('asc val files',tfrec_files_val_ls_asc)

In [ ]:
len_files_to_shuffle = (len(tfrec_files_ls_asc)) * 100 #the last one has something between one and hundred
len_files_to_shuffle

In [ ]:
len_files_val_to_shuffle = (len(tfrec_files_val_ls)) * 50 #the last one has something between one and hundred
len_files_val_to_shuffle

Can make the val dataset exactly as long as the batch size? such that there is no small remainder? No there are only 100...

#CGAN
https://machinelearningmastery.com/how-to-develop-a-pix2pix-gan-for-image-to-image-translation/

Alternative ; https://www.tensorflow.org/tutorials/generative/pix2pix#training

## Define cGAN

In [ ]:
# define the discriminator model
def define_discriminator(image_shape,image_shape_target):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape_target)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5) #lr
  
  # opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
  # model.compile(loss= binary_focal_dice_loss, optimizer=opt, loss_weights=[0.5]) #binary_focal_dice #metrics toevoegen as metrics = []
  


	return model

In [ ]:

# # define an encoder block
# def define_encoder_block(layer_in, n_filters, batchnorm=True):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# add downsampling layer
# 	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
# 	# conditionally add batch normalization
# 	if batchnorm:
# 		g = BatchNormalization()(g, training=True)
# 	# leaky relu activation
# 	g = LeakyReLU(alpha=0.2)(g)
# 	return g

# # define a decoder block
# def decoder_block(layer_in, skip_in, n_filters, dropout=True):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# add upsampling layer
# 	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
# 	# add batch normalization
# 	g = BatchNormalization()(g, training=True)
# 	# conditionally add dropout
# 	if dropout:
# 		g = Dropout(0.5)(g, training=True)
# 	# merge with skip connection
# 	g = Concatenate()([g, skip_in])
# 	# relu activation
# 	g = Activation('relu')(g)
# 	return g

# # define the standalone generator model
# def define_generator(image_shape=(256,256,3)):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# image input
# 	in_image = Input(shape=image_shape)
# 	# encoder model
# 	e1 = define_encoder_block(in_image, 64, batchnorm=False)
# 	e2 = define_encoder_block(e1, 128)
# 	e3 = define_encoder_block(e2, 256)
# 	e4 = define_encoder_block(e3, 512)
# 	e5 = define_encoder_block(e4, 512)
# 	e6 = define_encoder_block(e5, 512)
# 	e7 = define_encoder_block(e6, 512)
# 	# bottleneck, no batch norm and relu
# 	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
# 	b = Activation('relu')(b)
# 	# decoder model
# 	d1 = decoder_block(b, e7, 512)
# 	d2 = decoder_block(d1, e6, 512)
# 	d3 = decoder_block(d2, e5, 512)
# 	d4 = decoder_block(d3, e4, 512, dropout=False)
# 	d5 = decoder_block(d4, e3, 256, dropout=False)
# 	d6 = decoder_block(d5, e2, 128, dropout=False)
# 	d7 = decoder_block(d6, e1, 64, dropout=False)
# 	# output
# 	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
# 	out_image = Activation('tanh')(g) #change to sigmoid, then will be between 0 and 1 perfect for binary. 
# 	# define model
# 	model = Model(in_image, out_image)
# 	return model


In [ ]:

# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output, 1 band without dummy
	g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g) #change to sigmoid, then will be between 0 and 1 perfect for binary. But pix2pix says tanh
	# define model
	model = Model(in_image, out_image)
	return model


In [ ]:

# # define the combined generator and discriminator model, for updating the generator
# def define_gan(g_model, d_model, image_shape):
# 	# make weights in the discriminator not trainable
# 	for layer in d_model.layers:
# 		if not isinstance(layer, BatchNormalization):
# 			layer.trainable = False
# 	# define the source image
# 	in_src = Input(shape=image_shape)
# 	# connect the source image to the generator input
# 	gen_out = g_model(in_src)
# 	# connect the source input and generator output to the discriminator input
# 	dis_out = d_model([in_src, gen_out])
# 	# src image as input, generated image and classification output
# 	model = Model(in_src, [dis_out, gen_out])
# 	# compile model
# 	opt = Adam(learning_rate=0.0002, beta_1=0.5)
#   # opt = Adam(lr=0.0002, beta_1=0.5)
# 	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100]) #binary_focal_dice
# 	return model

In [ ]:

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
  # opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=[bce_dice_loss, 'mae'], optimizer=opt, loss_weights=[1,100]) #binary_focal_dice
	return model

## Load performance / Save model

In [ ]:
# def save_model(file_path_to_save,model,step):
#   filename = file_path_to_save + 'modelopt__%s.h5' % (step+1)
#   model.save(filename,include_optimizer = True) 
#   print('>Saved model: %s ' % (filename))


def save_model(file_path_to_save,model_name,model,step):
  filename = file_path_to_save + model_name + '__%s.h5' % (step+1)
  model.save(filename,include_optimizer = True) 
  print('>Saved model: %s ' % (filename))

Do a iou per epoch, and plot some specific images

In [ ]:
def iou_coe(output, target, axis,threshold=0.0, smooth=1e-5):
# def iou_coe(output, target, threshold=0.0, axis=(1,2,3), smooth=1e-5):
    """" With tanh threshold = 0.0
    """ 

    """Non-differentiable Intersection over Union (IoU) for comparing the
    similarity of two batch of data, usually be used for evaluating binary image segmentation.
    The coefficient between 0 to 1, and 1 means totally match.

    Parameters
    -----------
    output : tensor
        A batch of distribution with shape: [batch_size, ....], (any dimensions).
    target : tensor
        The target distribution, format the same with `output`.
    threshold : float
        The threshold value to be true.
    axis : tuple of integer
        All dimensions are reduced, default ``(1,2,3)``.
    smooth : float
        This small value will be added to the numerator and denominator, see ``dice_coe``.

    Notes
    ------
    - IoU cannot be used as training loss, people usually use dice coefficient for training, IoU and hard-dice for evaluating.

    """
    pre = tf.cast(output > threshold, dtype=tf.float32)
    truth = tf.cast(target > threshold, dtype=tf.float32)
    inse = tf.reduce_sum(tf.multiply(pre, truth), axis=axis)  # AND
    union = tf.reduce_sum(tf.cast(tf.add(pre, truth) >= 1, dtype=tf.float32), axis=axis)  # OR
    # old axis=[0,1,2,3]
    # epsilon = 1e-5
    # batch_iou = inse / (union + epsilon)
    # new haodong
    batch_iou = (inse + smooth) / (union + smooth)
    iou = tf.reduce_mean(batch_iou, name='iou_coe')
    return iou  # , pre, truth, inse, union

In [ ]:
def dice_hard_coe(output, target,  axis, threshold=0.0,smooth=1e-5):
# def dice_hard_coe(output, target, threshold=0.0, threshold_to_target = 0.0, axis=(1, 2, 3), smooth=1e-5):

    """" With tanh threshold = 0.0
    """ 


    """Non-differentiable Sørensen–Dice coefficient for comparing the similarity
    of two batch of data, usually be used for binary image segmentation i.e. labels are binary.
    The coefficient between 0 to 1, 1 if totally match.

    Parameters
    -----------
    output : tensor
        A distribution with shape: [batch_size, ....], (any dimensions).
    target : tensor
        The target distribution, format the same with `output`.
    threshold : float
        The threshold value to be true.
    axis : tuple of integer
        All dimensions are reduced, default ``(1,2,3)``.
    smooth : float
        This small value will be added to the numerator and denominator, see ``dice_coe``.

    References
    -----------
    - `Wiki-Dice <https://en.wikipedia.org/wiki/Sørensen–Dice_coefficient>`__

    """



    output = tf.cast(output > threshold, dtype=tf.float32)
    target = tf.cast(target > threshold, dtype=tf.float32) 
    inse = tf.reduce_sum(tf.multiply(output, target), axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)
    # old axis=[0,1,2,3]
    # hard_dice = 2 * (inse) / (l + r)
    # epsilon = 1e-5
    # hard_dice = tf.clip_by_value(hard_dice, 0, 1.0-epsilon)
    # new haodong
    hard_dice = (2. * inse + smooth) / (l + r + smooth)
    ##
    hard_dice = tf.reduce_mean(hard_dice, name='hard_dice')
    return hard_dice



## Defs of load_samples

In [ ]:

# select a batch of random samples, returns images and target
def generate_real_samples(batch):
  # unpack dataset
  X1,X2 = batch
  n_samples = X1.shape[0] 
	# generate 'real' class labels (1)
  #patch_shape is a constant
  y = ones((n_samples, patch_shape, patch_shape, 1))
  return [X1, X2], y


In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


## Run GAN model
Here need to define that you use TPU


In [ ]:

image_shape = (256,256,3)
image_shape_target = (256,256,1) #needs to be 256,256,1 not 256,256!


In [ ]:
with tpu_strategy.scope():
  d_model = define_discriminator(image_shape,image_shape_target)
  g_model = define_generator(image_shape)
  gan_model = define_gan(g_model,d_model,image_shape) 


In [ ]:
patch_shape = d_model.output_shape[1]

In [ ]:
# gan_model.summary()

In [ ]:
# d_model.summary()

In [ ]:
# g_model.summary()

# TF dataset

https://www.kaggle.com/docs/tpu
Could add some smart things from here

## Parse and decode tfrec

In [ ]:
#with image as string

def parse_record(record):
  tfrecord_format = {
    "Y": tf.io.FixedLenFeature([],tf.string),
    "sar1": tf.io.FixedLenFeature([],tf.string),
    "sar2": tf.io.FixedLenFeature([],tf.string),
    "dem": tf.io.FixedLenFeature([],tf.string)
   
    }
  return tf.io.parse_single_example(record, tfrecord_format) 

In [ ]:
dim = 256

In [ ]:
#return to image, so decode parse record
def decode_record(record):

  record_parsed = parse_record(record)
  # dataType = float32
  b1 = tf.io.decode_raw(
      record_parsed['sar1'],out_type = float
  )
  b2 = tf.io.decode_raw(
      record_parsed['sar2'],out_type = float
  )
  dem = tf.io.decode_raw(
      record_parsed['dem'],out_type = float
  )
  y = tf.io.decode_raw(
      record_parsed['Y'],out_type = float      
  )

# sometimes values slightly below -1 are in the dem band, filter them to -1. (these are dem values which where below 0 before scaling apparently) 
  # value = tf.constant(-1.0,dtype=float)
  dem = tf.where([dem < -1.000], x=[-1.000], y = [dem])#doesnt work?
  # tf.where(dem < value, x=value, y = dem)

  # #images are flattened, so have to put back to og shape
  b1 = tf.reshape(b1,(dim,dim))  #w, h 256,256
  b2 = tf.reshape(b2,(dim,dim))
  dem = tf.reshape(dem,(dim,dim))
  y1 = tf.reshape(y,(dim,dim,1)) #hopefully y1 is then (256,256,1)

  # #might want to stack them before
  image_x = tf.stack((b1,b2,dem),axis = -1) #if axis = -1 then get (nr im,w,h,3)

  # y2 = tf.experimental.numpy.full_like(y1,-1,dtype= tf.float32)
  # y3 = tf.experimental.numpy.full_like(y1,-1,dtype= tf.float32)
  # # y2 = tf.experimental.numpy.full_like(y1,0,dtype= tf.float32)
  # # y3 = tf.experimental.numpy.full_like(y1,0,dtype= tf.float32)
  # image_y = tf.stack((y1,y2,y3),axis = -1)

  # return image_x,image_y
  return image_x,y1


## Add augmentations

In [ ]:
def dropout(image, size=256, p = 0.75, ct = 12, sz = 0.05):
    # input - one image of size [size, size, 3] not a batch of [b, size, size, 3]
    # output - image with ct squares of side size sz*size removed
    p = tf.cast(tf.random.uniform([],0,1) < p, tf.int32)
    if (p == 0)|(ct == 0)|(sz == 0): return image

    for k in range(ct):
        # choose random location
        x = tf.cast(tf.random.uniform([], 0, size),tf.int32)
        y = tf.cast(tf.random.uniform([], 0, size),tf.int32)
        
        # compute square
        width = tf.cast(sz*size, tf.int32)*p
        ya = tf.math.maximum(0, y-width//2)
        yb = tf.math.minimum(size, y+width//2)
        xa = tf.math.maximum(0, x-width//2)
        xb = tf.math.minimum(size, x+width//2)
        
        # dropout image
        one = image[ya:yb, 0:xa, :]
        two = tf.zeros([yb-ya, xb-xa, image.shape[-1]]) 
        three = image[ya:yb, xb:size, :]
        middle = tf.concat([one, two, three], axis=1)
        image = tf.concat([image[0:ya, :, :], middle, image[yb:size, :, :]], axis=0)

    return image


In [ ]:

def augment(image, mask, seed=P['SEED'],size=P['SIZE']): #not sure if input image,mask is okinstead of record..
    # Flip
    if tf.random.uniform([], seed=seed) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)

    if tf.random.uniform([],seed=seed) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)
        
    if tf.random.uniform([],seed=seed) > .75:
        image = tf.image.transpose(image)
        mask = tf.image.transpose(mask)
      
    p_rotate = tf.random.uniform([],seed=seed)
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
        mask = tf.image.rot90(mask, k=3)
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
        mask = tf.image.rot90(mask, k=2)
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        mask = tf.image.rot90(mask, k=1)
        
    # Drop out. Only for the image with 3 bands
    # image = dropout(image, size, p=1.0, ct=10, sz=0.1)
                
    
    
    return image, mask




## Create data set from tfrec

In [ ]:
AUTO = tf.data.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

def get_training_dataset(files, len_shuffle,seed=P['SEED'], batch_size=n_batch):
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.cache()
    dataset = dataset.repeat(2)
    dataset = dataset.map(lambda ex: decode_record(ex), num_parallel_calls=AUTO)
    dataset = dataset.map(augment, num_parallel_calls=AUTO) #maybe should do lambda x y here..
    dataset = dataset.shuffle(buffer_size= len_shuffle, seed=seed) #when shuffle??
    dataset = dataset.batch(batch_size, drop_remainder=False) #bc seconde one is only 16 for bsize 512.
    dataset = dataset.prefetch(AUTO)
    
    return dataset


In [ ]:
def count_examples(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(str(filename)).group(1)) for filename in filenames]
    
    return np.sum(n)

In [ ]:
##somehow this function doesnt work.. maybe bc drop remainder!!!! --> bc val ls < batch size 
def get_validation_dataset(files, len_shuffle,ordered=True,seed=P['SEED'], batch_size=n_batch):
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.cache()
    # dataset = dataset.map(lambda ex: decode_record(ex), num_parallel_calls=AUTO)
    dataset = dataset.map(decode_record, num_parallel_calls=AUTO)
    dataset = dataset.shuffle(buffer_size = len_shuffle, seed=seed) 
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.prefetch(AUTO)

    return dataset



## get Kaggle gs, create ds

for training set

In [ ]:
# train_dataset = get_training_dataset(tfrec_files_all_ls,700)
# train_dataset = get_training_dataset(tfrec_files_all_ls_des,450)
# train_dataset = get_training_dataset(tfrec_files_ls_asc,len_files_to_shuffle)
# train_dataset = get_training_dataset(tfrec_files_ls_des,200)
train_dataset = get_training_dataset(tfrec_files_ls,400)
# tfrec_files_ls_des

In [ ]:
val_dataset = get_validation_dataset(tfrec_files_val_ls,len_files_val_to_shuffle) 
# val_dataset = get_validation_dataset(files = tfrec_files_val_ls_asc,len_shuffle = 200) #what buffer size??

### Check if correct

In [ ]:
#dropout in every image?? Lets try without first
def check_y(ds):
  for [x1,x2] in ds.take(1):
    tar = x2[:,:,:,0]
    b1 = x1[:,:,:,0]
    b2 = x1[:,:,:,1]
    dem = x1[:,:,:,2]
    print(tar.shape[0])
    # for i in range(tar.shape[0]):
    for i in range(10):
      plt.imshow(b1[i])
      plt.axis('off')
      plt.show()
      plt.close()
      plt.imshow(b2[i])
      plt.axis('off')
      plt.show()
      plt.close()
      plt.imshow(dem[i])
      plt.axis('off')
      plt.show()
      plt.close()
      plt.imshow(tar[i])
      plt.axis('off')
      plt.show()
      plt.close()

In [ ]:
check_y(train_dataset)

# Train

## Train

Maybe add some stuff to stop training if model is shit -->
- if iou and dice of test are still nan after >10 epochs
- if loss of dis. is still 0/close to 0 after>10 epochs (gan converge failure)


https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/

The loss is reported each training iteration, including the discriminator loss on real examples (d1), discriminator loss on generated or fake examples (d2), and generator loss, which is a weighted average of adversarial and L1 loss (g).


This is important. A stable GAN will have a discriminator loss around 0.5, typically between 0.5 and maybe as high as 0.7 or 0.8. The generator loss is typically higher and may hover around 1.0, 1.5, 2.0, or even higher

This is an example of the normal or expected loss during training. Namely, discriminator loss for real and fake samples is about the same at or around 0.5, and loss for the generator is slightly higher between 0.5 and 2.0

So quit training after x epochs if d1 and/or d2 loss is close to 0. Also if iou remains nan..

Acutally its possible to directly do iou over batch instead of in for loop I think...

In [ ]:
path_to_save

In [ ]:
path_results

In [ ]:
def train(batch_train,batch_test,d_model=d_model, g_model=g_model, gan_model=gan_model,num_epochs = 100,epoch_stop =30,epoch_save=[49,59,69,79,89,99] ,file_path_to_save_model=path_to_save,file_path_to_save_result=path_results):

  """" train model
  Here also loss is calculated
  Also stops after dloss remains low and/or iou remains nan (aka 0/x ? what gives a nan value? )
  Here only the iou and dice for the positive images is calculated
  batch_train: the train dataset, tfrecord dataset which is batched
  batch_test: the test dataset, tfrecord dataset which is batched, only ls images
  No if condition on validation set since the validation is only ls images
  """""
  
  #dice and iou over epochs for train and test set.
  iou_over_epochs,iou_test_over_epochs = list(), list()
  d1_hist, d2_hist, g_hist = list(), list(), list()

  for epoch in tqdm(range(num_epochs)):

    #metric per image over all image in one epoch
    iou_over_epoch,iou_test_over_epoch = list(), list()
    d1_epoch, d2_epoch, g_epoch = list(),list(),list()

    for [x1,x2] in tqdm(batch_train):

      # select a batch of real samples
      [src, target], y_real = generate_real_samples([x1,x2]) 
      # generate a batch of fake samples
      gen_target, y_fake = generate_fake_samples(g_model, src)  
      # update discriminator for real samples
      d_loss1 = d_model.train_on_batch([src,target], y_real)
       # update discriminator for generated samples
      d_loss2 = d_model.train_on_batch([src, gen_target], y_fake)
      # update the generator
      g_loss, _, _ = gan_model.train_on_batch(src, [y_real, target])

      
      #get 0 band!
      iou_over_batch = iou_coe(gen_target[:,:,:,0],target[:,:,:,0],axis=(1,2))

      #record loss history, to get a plot of converging (hopefully) dis & gen loss over epochs (only for train ds(?))
      d1_epoch.append(d_loss1)
      d2_epoch.append(d_loss2)
      g_epoch.append(g_loss)
  
      
      #save iou
      iou_over_epoch.append(iou_over_batch)


#retracing might be bc the batch_test has other size bc its smaller, so dif batch sizes..
    for [x1,x2] in tqdm(batch_test):
      target_pred = g_model.predict(x1)
      iou_test = iou_coe(target_pred[:,:,:,0],x2[:,:,:,0],axis=(1,2))
      #save iou
      iou_test_over_epoch.append(iou_test)


    #  #save the metric accross epochs.
    iou_test_over_epochs.append(tf.reduce_mean(iou_test_over_epoch))
    #save the metric accross epochs, for train
    iou_over_epochs.append(tf.reduce_mean(iou_over_epoch))
         

    # out of batch loop, in epoch loop
    # record history over epochs --> could either do mean of each epoch or just leave them for each batch.
    d1_hist.append(d1_epoch)
    d2_hist.append(d2_epoch)
    g_hist.append(g_epoch)

    
    if tf.equal(epoch,epoch_stop): #this works!
      print('d loss at epoch stop',d_loss1)
      # print(d_loss1 == np.nan)
      # print(d_loss1 == float('nan'))

      if tf.math.less(d_loss1,tf.constant(0.01)):
        print('then should quit since d1 loss is close to zero:',d_loss1) #works
        # break
      elif np.isnan(d_loss1):
        print("its nan")
        # break
  

    # if tf.math.greater_equal(epoch,epoch_save): #if want to retrain might do it at every tenth model starting at 50..
      #if want to retrain (e.g. further tha epoch 100 such as 200, than need to save all three models)
    if epoch in epoch_save:
      # Better save both as GAN is the orchestrated work of both the generator and discriminator together. --> seems like only gan?
      save_model(file_path_to_save_model,'g_model',g_model,epoch)
      save_model(file_path_to_save_model,'d_model',d_model,epoch)
      save_model(file_path_to_save_model,'gan_model',gan_model,epoch)


  #not sure if this would be too much for memory..
  #save losses
  fn_d1 = file_path_to_save_result + 'd1.txt'
  np.savetxt(fn_d1, d1_hist) #these are nested lists!
  fn_d2 = file_path_to_save_result + 'd2.txt'
  np.savetxt(fn_d2, d2_hist)
  fn_g = file_path_to_save_result + 'g.txt'
  np.savetxt(fn_g, g_hist)


  #save dice and iou as well
  fn_iou = file_path_to_save_result+'iou_over_epochs.txt'
  np.savetxt(fn_iou, iou_over_epochs)

  fn_iou = file_path_to_save_result+'iou_test_over_epochs.txt' 
  np.savetxt(fn_iou, iou_test_over_epochs)



In [ ]:
train(train_dataset,val_dataset)  #3:18 28% for bsize 8 with repeat 2 shit.... Maybe gpu is faster than tpy if you only use bsize 8?
#65 percent, 7:41h
# 132 batches per epoch, sure I didnt accidentally put repeat(3)?
#70 8:17h
#quit at 72 percent at 8:31 :(((((

# Retrain loaded model

## load model

In [ ]:
# example of loading a pix2pix model and using it for image to image translation
from keras.models import load_model
from numpy import load
from numpy import vstack
from numpy.random import randint

!ls -la {path_to_save}

fn_models = [str(f) for f in pathlib.Path(path_to_save).glob('*.h5')]
fn_models

In [ ]:
# fn_d_model = path_to_save + "d_model__100.h5"
# fn_g_model = path_to_save + "g_model__100.h5"
fn_gan_model = fn_models[-1]
fn_d_model = fn_models[-2]
fn_g_model = fn_models[-3]

print("the models are",fn_gan_model,fn_d_model,fn_g_model)

In [ ]:


epoch_loaded_model = int(fn_gan_model.split("__",1)[-1].split(".h5")[0]) #but then al should have names with __
epoch_loaded_model

# But then should have the gan, gen and dis...

with tpu_strategy.scope():
    # load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    d_model = tf.keras.models.load_model(fn_d_model)
    g_model = tf.keras.models.load_model(fn_g_model)
    gan_model = tf.keras.models.load_model(fn_gan_model)#,custom_objects = {'binary_focal_loss_plus_dice_loss':binary_focal_dice_loss}, options=load_locally) # loading in Tensorflow's "SavedModel" format

d_model.summary()

g_model.summary()

gan_model.summary()

## Run train on reloaded model

new path to save results and model to..
add real epoch nr for use for file name f saved model. ad possibly in results name


In [ ]:

def train(batch_train,batch_test,d_model=d_model, g_model=g_model, gan_model=gan_model,num_epochs = 80,epoch_stop =30,epoch_begin_rerun = epoch_loaded_model,epoch_save=[49,59,69,79,89,99] ,file_path_to_save_model=path_to_save,file_path_to_save_result=path_results):

  """" train model
  Here also loss is calculated
  Also stops after dloss remains low and/or iou remains nan (aka 0/x ? what gives a nan value? )
  Here only the iou and dice for the positive images is calculated
  batch_train: the train dataset, tfrecord dataset which is batched
  batch_test: the test dataset, tfrecord dataset which is batched, only ls images
  No if condition on validation set since the validation is only ls images
  """""

  file_path_to_save_model = file_path_to_save_model + "rerun1/"
  file_path_to_save_result = file_path_to_save_result + "rerun1/"
  
  #dice and iou over epochs for train and test set.
  iou_over_epochs,iou_test_over_epochs = list(), list()
  d1_hist, d2_hist, g_hist = list(), list(), list()

  for epoch in tqdm(range(num_epochs)):

    #metric per image over all image in one epoch
    iou_over_epoch,iou_test_over_epoch = list(), list()
    d1_epoch, d2_epoch, g_epoch = list(),list(),list()

    for [x1,x2] in tqdm(batch_train):

      # select a batch of real samples
      [src, target], y_real = generate_real_samples([x1,x2]) 
      # generate a batch of fake samples
      gen_target, y_fake = generate_fake_samples(g_model, src)  
      # update discriminator for real samples
      d_loss1 = d_model.train_on_batch([src,target], y_real)
       # update discriminator for generated samples
      d_loss2 = d_model.train_on_batch([src, gen_target], y_fake)
      # update the generator
      g_loss, _, _ = gan_model.train_on_batch(src, [y_real, target])


      #record loss history, to get a plot of converging (hopefully) dis & gen loss over epochs (only for train ds(?))
      d1_epoch.append(d_loss1)
      d2_epoch.append(d_loss2)
      g_epoch.append(g_loss)
    

      #iou with condition
      iou_over_batch = iou_coe(gen_target[:,:,:,0],target[:,:,:,0],axis=(1,2))
    
      #save iou
      iou_over_epoch.append(iou_over_batch)
  
#retracing might be bc the batch_test has other size bc its smaller, so dif batch sizes..
    for [x1,x2] in tqdm(batch_test):
      target_pred = g_model.predict(x1)

      iou_over_batch = iou_coe(target_pred[:,:,:,0],x2[:,:,:,0],axis=(1,2))
       
      #save iou
      iou_test_over_epoch.append(iou_over_batch)


    #  #save the metric accross epochs.
    iou_test_over_epochs.append(tf.reduce_mean(iou_test_over_epoch))
    iou_over_epochs.append(tf.reduce_mean(iou_over_epoch))
    
         

    # out of batch loop, in epoch loop
    # record history over epochs --> could either do mean of each epoch or just leave them for each batch.
    d1_hist.append(d1_epoch)
    d2_hist.append(d2_epoch)
    g_hist.append(g_epoch)

  
    if tf.equal(epoch,epoch_stop): #this works!
      print('d loss at epoch stop',d_loss1)
      # print(d_loss1 == np.nan)
      # print(d_loss1 == float('nan'))

      if tf.math.less(d_loss1,tf.constant(0.01)):
        print('then should quit since d1 loss is close to zero:',d_loss1) #works
        # break
      elif np.isnan(d_loss1):
        print("its nan")
        # break
  

    # if tf.math.greater_equal(epoch,epoch_save): #if want to retrain might do it at every tenth model starting at 50..
      #if want to retrain (e.g. further tha epoch 100 such as 200, than need to save all three models)
    if epoch in epoch_save:
      epoch_rerun = epoch + epoch_begin_rerun 
      # Better save both as GAN is the orchestrated work of both the generator and discriminator together. --> seems like only gan?
      save_model(file_path_to_save_model,"g_model",g_model,epoch_rerun)
      save_model(file_path_to_save_model,"d_model",d_model,epoch_rerun)
      save_model(file_path_to_save_model,"gan_model",gan_model,epoch_rerun)




  #not sure if this would be too much for memory..
  #save losses
  fn_d1 = file_path_to_save_result + 'd1.txt'
  np.savetxt(fn_d1, d1_hist) #these are nested lists!
  fn_d2 = file_path_to_save_result + 'd2.txt'
  np.savetxt(fn_d2, d2_hist)
  fn_g = file_path_to_save_result + 'g.txt'
  np.savetxt(fn_g, g_hist)


  #save  iou as well
  fn_iou = file_path_to_save_result+'iou_over_epochs.txt'
  np.savetxt(fn_iou, iou_over_epochs)

  fn_iou = file_path_to_save_result+'iou_test_over_epochs.txt' 
  np.savetxt(fn_iou, iou_test_over_epochs)






In [ ]:

train(train_dataset,val_dataset) 